In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
rows = x.shape[0] 
cols = x.shape[1] * x.shape[2]
x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 1ms/step


In [2]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42, max_iter=1000)

In [3]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [4]:
model.coef_

array([[-1.17628726e+00,  1.24389930e+00,  5.57148173e-01,
         3.27721864e-01,  2.24072045e-01,  1.16421082e+00,
         4.91213182e-01, -1.73601102e-01,  5.95890754e-01,
         7.69588909e-01,  3.33183128e-01,  1.84413810e+00,
        -4.04278839e+00, -4.30159837e+00,  2.80382065e+00,
         2.08907153e+00,  3.82958940e+00, -6.68605340e-01,
         3.75917940e+00, -6.39223646e+00, -7.42880504e-01,
         9.14415427e-01,  7.28928349e-01, -3.22762220e-01,
        -1.26350039e+00, -3.66931328e-01,  1.39592957e+00,
         7.98388642e-01,  2.11780183e+00, -3.20047599e+00,
        -7.81251819e-01,  2.44257677e-02, -1.70756007e-01,
        -4.27949558e-01,  3.30008298e-01, -5.85718239e-01,
         9.96355990e-01,  1.28172173e+00,  5.51819017e-01,
        -9.65275497e-02, -9.68296975e-02,  1.29283316e-01,
         1.40017828e-01, -5.76528381e-01,  1.33568978e-03,
        -1.36864139e+00,  1.13026080e+00,  3.68616978e-01,
         7.78798390e-01, -5.68300075e-01, -8.96628703e-0

In [5]:
model.intercept_

array([-0.25258559,  0.67913028, -0.42654468])

In [6]:
model.score(x_test,y_test)

0.6984126984126984

In [10]:
model.score(x_train,y_train)

0.6979768786127167

In [7]:
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , roc_auc_score
def Evaluation_metrics(actual,pred):
    accuracy_sco = accuracy_score(actual,pred)
    confusion_mat = confusion_matrix(actual,pred)
    precision_sco = precision_score(actual,pred,average='macro')
    recall_sco = recall_score(actual,pred,average='macro')
    f1_sco = f1_score(actual,pred,average='macro')
    return accuracy_sco,confusion_mat,precision_sco,recall_sco,f1_sco

In [8]:
predicted_val = model.predict(x_test)

In [9]:
Evaluation_metrics(y_test,predicted_val)

(0.6984126984126984,
 array([[178,  26,  28],
        [ 15, 134,  82],
        [ 32,  26, 172]], dtype=int64),
 0.7071567655507257,
 0.6983846821178156,
 0.6978514289225076)